In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Replace with your actual file path
file_path = '/content/drive/MyDrive/Data Mining/Main_Assignment_Shared_resources/Health_and_Personal_Care.jsonl'

# Load the .jsonl file into a pandas DataFrame
df = pd.read_json(file_path, lines=True)

# Display the first 5 rows of the DataFrame
print("DataFrame Loaded Successfully!")
df.head()


DataFrame Loaded Successfully!


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True


In [ ]:
# number of reviews
print(len(df))

print(list(df.columns))

494121
['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']


In [ ]:
# count of all ratings to check if there's imbalance
df['rating'].value_counts()

,count
rating,
5,301713
1,69564
4,57000
3,36949
2,28895


In [ ]:
df.isnull().sum()

,0
rating,0
title,0
text,0
images,0
asin,0
parent_asin,0
user_id,0
timestamp,0
helpful_vote,0
verified_purchase,0


In [ ]:
# Imports related to data loading and preprocessing
!pip install --upgrade --force-reinstall numpy scipy gensim
import pandas as pd
import numpy as np
import re
import nltk
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Imports related to Word2Vec
import gensim.downloader as api

# Imports related to Deep Learning Model (TensorFlow/Keras)
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
# Try importing the legacy Adam optimizer to handle decay, fallback if it doesn't exist
try:
    from tensorflow.keras.optimizers.legacy import Adam
    print("Using legacy Adam optimizer.")
except ImportError:
    from tensorflow.keras.optimizers import Adam
    print("Using standard Adam optimizer (decay argument might be ignored).")

# Ensure numpy is comprehensively imported (already done above, but good practice)
import numpy as np

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64

# Preprocessing

In [ ]:
df.dropna(subset=['text', 'rating'], inplace=True)
print(f"\nDataFrame shape after dropping rows with missing text/rating: {df.shape}")

# Combine 'title' and 'text'
# Fill missing titles with an empty string BEFORE concatenation
df['title'] = df['title'].fillna('')
df['review_full'] = df['title'] + ' ' + df['text']

# Text Cleaning Setup
nltk.download('punkt_tab', quiet=True) # Download the specific resource needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. Lowercasing
    text = text.lower()
    # 2. Remove HTML tags (if any)
    text = re.sub(r'<.*?>', '', text)
    # 3. Remove punctuation and special characters (keeping only letters and whitespace)
    text = re.sub(r'[^a-z\s]', '', text)
    # 4. Tokenization
    tokens = word_tokenize(text)
    # 5. Remove Stop Words and Lemmatize
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and len(word) > 1] # Keep words with length > 1
    # 6. Join back into string
    return ' '.join(cleaned_tokens)

print("\nStarting text cleaning (this may take a while)...")
# Apply cleaning function - Monitor progress if needed for large datasets
# Consider df['review_full'].parallel_apply(clean_text) using libraries like pandarallel if speed is critical
df['review_cleaned'] = df['review_full'].apply(clean_text)
print("Text cleaning completed.")

# Drop rows where cleaning might have resulted in empty strings
df.dropna(subset=['review_cleaned'], inplace=True)
df = df[df['review_cleaned'].str.strip() != ''] # Ensure no empty strings after cleaning
print(f"DataFrame shape after cleaning and removing empty reviews: {df.shape}")


DataFrame shape after dropping rows with missing text/rating: (494121, 10)

Starting text cleaning (this may take a while)...
Text cleaning completed.
DataFrame shape after cleaning and removing empty reviews: (493820, 12)


In [ ]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,review_full,review_cleaned
0,4,12 mg is 12 on the periodic table people! Mg f...,This review is more to clarify someone else’s ...,[],B07TDSJZMR,B07TDSJZMR,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-02-06 00:49:35.902,3,True,12 mg is 12 on the periodic table people! Mg f...,mg periodic table people mg magnesium review c...
1,5,Save the lanet using less plastic.,Love these easy multitasking bleach tablets. B...,[],B08637FWWF,B08637FWWF,AEVWAM3YWN5URJVJIZZ6XPD2MKIA,2020-11-02 22:03:06.880,3,True,Save the lanet using less plastic. Love these ...,save lanet using less plastic love easy multit...
2,5,Fantastic,I have been suffering a couple months with hee...,[],B07KJVGNN5,B07KJVGNN5,AHSPLDNW5OOUK2PLH7GXLACFBZNQ,2019-07-24 11:13:58.905,0,True,Fantastic I have been suffering a couple month...,fantastic suffering couple month heel pain pla...
3,4,It holds the water and makes bubbles. That's ...,"It's cheap and it does what I wanted. The ""ma...",[],B007HY7GC2,B092RP73CX,AEZGPLOYTSAPR3DHZKKXEFPAXUAA,2022-09-04 02:29:02.725,7,True,It holds the water and makes bubbles. That's ...,hold water make bubble thats bought cheap want...
4,1,Not for me,Didn't do a thing for me. Not saying they don'...,[],B08KYJLF5T,B08KYJLF5T,AEQAYV7RXZEBXMQIQPL6KCT2CFWQ,2022-01-20 23:53:07.262,0,True,Not for me Didn't do a thing for me. Not sayin...,didnt thing saying dont


# Download and Load Word2Vec Model

In [ ]:
# Download and Load Word2Vec Model (e.g., Google News 300d)
# This requires internet connection and might take some time to download (~1.6 GB)
print("Loading pre-trained Word2Vec model (word2vec-google-news-300)...")
try:
    # Check if model is already downloaded (optional, gensim might cache it)
    # model_path = api.load('word2vec-google-news-300', return_path=True)
    # print(f"Model already available at: {model_path}")
    # wv = api.load('word2vec-google-news-300') # Just load it

    # Simpler approach: just try loading, gensim handles download/cache
    wv = api.load('word2vec-google-news-300')
    print("Word2Vec model loaded successfully.")
    # Example: Get vector for a word
    # print("Example vector for 'king':", wv['king'][:10]) # Print first 10 dims
    print(f"Model has {len(wv.index_to_key)} words and vector size {wv.vector_size}")

except Exception as e:
    print(f"Error loading Word2Vec model: {e}")
    print("Please ensure you have an internet connection and sufficient disk space.")
    wv = None # Set wv to None if loading fails

Loading pre-trained Word2Vec model (word2vec-google-news-300)...
[==================================================] 100.0% 1662.8/1662.8MB downloaded
Word2Vec model loaded successfully.
Model has 3000000 words and vector size 300


# Train/Split Data

In [ ]:
# Extract features and labels
X_texts = df['review_cleaned'].values  # the preprocessed reviews
y = df['rating'].values       # the corresponding labels (ratings 1-5)

# Split the data
X_train_texts, X_test_texts, y_train_orig, y_test_orig = train_test_split(
    X_texts,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Number of training samples:", len(X_train_texts))
print("Number of testing samples:", len(X_test_texts))
# Note: y_train_orig and y_test_orig still hold ratings 1-5

Number of training samples: 395056
Number of testing samples: 98764


# Tokenization and Padding

In [ ]:
# Define maximum number of words to consider as features
MAX_NUM_WORDS = 20000 # You can adjust this based on your vocabulary size observation
# Define maximum length of sequence (reviews)
MAX_SEQUENCE_LENGTH = 150 # Adjust based on review length analysis if needed

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train_texts) # Fit only on training data

X_train_sequences = tokenizer.texts_to_sequences(X_train_texts)
X_test_sequences = tokenizer.texts_to_sequences(X_test_texts)

word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')

# --- Padding ---
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

print('Shape of training data tensor:', X_train_padded.shape)
print('Shape of testing data tensor:', X_test_padded.shape)

Found 149458 unique tokens.
Shape of training data tensor: (395056, 150)
Shape of testing data tensor: (98764, 150)


# Create Word2Vec Embedding Matrix

In [ ]:
# --- Prepare Embedding Matrix using Word2Vec ---
EMBEDDING_DIM = 300 # Word2Vec dimension requested
num_words = min(MAX_NUM_WORDS, len(word_index) + 1) # +1 because index 0 is reserved

embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
words_found = 0
words_not_found = 0

if wv is not None: # Proceed only if Word2Vec model loaded successfully
    for word, i in word_index.items():
        if i >= MAX_NUM_WORDS:
            continue # Skip words beyond the limit
        try:
            embedding_vector = wv[word] # Get vector from loaded Word2Vec model
            embedding_matrix[i] = embedding_vector
            words_found += 1
        except KeyError:
            # Words not found in Word2Vec vocab will be all-zeros.
            words_not_found += 1
            pass # embedding_matrix[i] remains zero

    print(f"Embedding Matrix shape: {embedding_matrix.shape}")
    print(f"Found Word2Vec embeddings for {words_found} out of {num_words-1} words in the tokenizer vocabulary.") # num_words includes padding/OOV
    print(f"Words not found in Word2Vec: {words_not_found}")
else:
    print("Word2Vec model (wv) not loaded. Cannot create embedding matrix.")
    # Handle this case - maybe stop execution or use random embeddings
    embedding_matrix = None # Indicate failure

Embedding Matrix shape: (20000, 300)
Found Word2Vec embeddings for 17304 out of 19999 words in the tokenizer vocabulary.
Words not found in Word2Vec: 2695


# Adjust Labels and Define num_classes

In [ ]:
y_train = y_train_orig - 1
y_test = y_test_orig - 1

# Determine the number of classes BASED ON THE USER REQUEST
# *** WARNING: User requested 9 output neurons, which mismatches the 5 rating classes (1-5 or 0-4). ***
# This will affect model training and interpretation. The model will predict one of 9 classes (0-8).
num_classes_requested = 9
actual_num_classes = len(np.unique(y_train_orig)) # This should be 5

print(f"Actual number of unique rating classes in data: {actual_num_classes}")
print(f"Number of output neurons requested: {num_classes_requested}")
print("Labels adjusted to 0-based indexing.")
print("Sample original labels:", y_train_orig[:5])
print("Sample adjusted labels (used for training):", y_train[:5])

# Use the requested number of classes for the model's output layer
num_classes = num_classes_requested

Actual number of unique rating classes in data: 5
Number of output neurons requested: 9
Labels adjusted to 0-based indexing.
Sample original labels: [5 5 2 1 4]
Sample adjusted labels (used for training): [4 4 1 0 3]


# Define BiLSTM Model Architecture

In [ ]:
# --- Build the BiLSTM Model ---

# Check if embedding matrix was created successfully
if embedding_matrix is None:
    raise ValueError("Embedding matrix was not created. Cannot build model.")

model = Sequential()

# Embedding Layer - Initialized with Word2Vec, not trainable
model.add(Embedding(input_dim=num_words, # Size of the vocabulary (limited by MAX_NUM_WORDS)
                    output_dim=EMBEDDING_DIM, # Dimension of Word2Vec (300)
                    weights=[embedding_matrix], # Pre-trained Word2Vec weights
                    input_length=MAX_SEQUENCE_LENGTH, # Length of input sequences
                    trainable=False)) # Freeze Word2Vec weights

# BiLSTM Layer 1
model.add(Bidirectional(LSTM(100, return_sequences=True))) # 100 neurons, return sequences for the next LSTM layer
model.add(Dropout(0.1)) # Dropout rate 0.1

# BiLSTM Layer 2
model.add(Bidirectional(LSTM(200))) # 200 neurons, default return_sequences=False
model.add(Dropout(0.1)) # Dropout rate 0.1

# Output Layer
# *** Using num_classes = 9 as requested, with softmax activation ***
model.add(Dense(num_classes, activation='softmax')) # Output layer for classification

# Print model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     6,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,000,000 (22.89 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 6,000,000 (22.89 MB)

# Compile the Mode

In [ ]:
# --- Compile the Model ---

# Define the optimizer with specified parameters
learning_rate = 1e-04
# decay_rate = 0.01 # Decay is deprecated in standard Keras 3 Adam

# Use the standard Keras 3 Adam optimizer.
# It does not accept the 'decay' argument directly.
# For learning rate reduction over time, use LearningRateScheduler callbacks or schedules.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
print(f"Using standard Keras 3 Adam optimizer with lr={learning_rate}.")
print("Note: 'decay' argument is not used. Consider Learning Rate Schedules if decay effect is desired.")


# Compile the model
model.compile(loss='sparse_categorical_crossentropy', # Use sparse CE because labels are integers (0-4)
              optimizer=optimizer,
              metrics=['accuracy']) # Measures how often prediction matches the integer label

print("Model compiled successfully.")

Using standard Keras 3 Adam optimizer with lr=0.0001.
Note: 'decay' argument is not used. Consider Learning Rate Schedules if decay effect is desired.
Model compiled successfully.


# Train the Model

In [ ]:
# --- Train the Model ---

BATCH_SIZE = 200
EPOCHS = 10

# Ensure data is ready before training
if X_train_padded is None or y_train is None or X_test_padded is None or y_test is None:
     raise ValueError("Training/Testing data is not ready. Check previous steps.")
if model is None:
    raise ValueError("Model is not defined. Check previous steps.")


print("Starting model training...")
history = model.fit(X_train_padded, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_test_padded, y_test),
                    verbose=1) # Set verbose=1 or 2 to see progress per epoch

print("Model training completed.")

# You can plot the history (accuracy/loss) after training if desired
# import matplotlib.pyplot as plt
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# ... etc.

Starting model training...
Epoch 1/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 227s 111ms/step - accuracy: 0.6632 - loss: 1.0287 - val_accuracy: 0.7297 - val_loss: 0.7598
Epoch 2/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 261s 111ms/step - accuracy: 0.7366 - loss: 0.7355 - val_accuracy: 0.7482 - val_loss: 0.6987
Epoch 3/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 262s 112ms/step - accuracy: 0.7515 - loss: 0.6904 - val_accuracy: 0.7540 - val_loss: 0.6778
Epoch 4/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 262s 112ms/step - accuracy: 0.7605 - loss: 0.6639 - val_accuracy: 0.7584 - val_loss: 0.6641
Epoch 5/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 262s 112ms/step - accuracy: 0.7626 - loss: 0.6539 - val_accuracy: 0.7585 - val_loss: 0.6628
Epoch 6/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 262s 112ms/step - accuracy: 0.7670 - loss: 0.6417 - val_accuracy: 0.7592 - val_loss: 0.6614
Epoch 7/10
1976/1976 ━━━━━━━━━━━━━━━━━━━━ 262s 112ms/step - accuracy: 0.7686 - loss: 0.6343 - val_accuracy: 0.7683 - val_loss: 0.6364
Epoch 8/10
1976/1976 ━━━━━━━━━━━━━━

# Evaluate the Model

In [ ]:
# --- Evaluate the Model ---

print("\nEvaluating model on the test set...")
loss, accuracy = model.evaluate(X_test_padded, y_test, batch_size=BATCH_SIZE, verbose=0)
print(f'Test Loss: {loss:.4f}')
# Accuracy here reflects how often the predicted class (0-8) matches the true label (0-4)
print(f'Test Accuracy (Keras evaluate): {accuracy:.4f}')

# --- Get Predictions and Detailed Report ---
print("\nGenerating predictions...")
y_pred_probs = model.predict(X_test_padded, batch_size=BATCH_SIZE)
# y_pred_classes will contain values from 0 to 8 (since num_classes=9)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

print("\n--- Evaluation Warning ---")
print("The model was trained with 9 output neurons as requested.")
print("True labels (y_test) are in the range 0-4.")
print("Predicted classes (y_pred_classes) are in the range 0-8.")
print("Direct comparison using metrics like classification report or standard confusion matrix might be misleading.")
print("--------------------------")

# Calculate accuracy using sklearn comparing predicted (0-8) vs true (0-4)
# This score will likely differ from Keras's accuracy if predictions fall outside 0-4
sklearn_accuracy = accuracy_score(y_test, y_pred_classes)
print(f'\nSklearn Accuracy Score (y_test vs y_pred_classes 0-8): {sklearn_accuracy:.4f}')


print('\nConfusion Matrix (y_test [0-4] vs y_pred_classes [0-8]):')
# Note: This matrix might be hard to interpret directly due to dimension mismatch.
# It shows where the true classes (rows 0-4) landed among the predicted classes (columns 0-8).
# You might need a custom mapping or interpretation based on why 9 outputs were chosen.
try:
    # Generate confusion matrix. sklearn handles shape mismatches by showing all predicted columns.
    cm = confusion_matrix(y_test, y_pred_classes)
    print(cm)
    print(f"Confusion Matrix Shape: {cm.shape}") # Will likely be (5, N) where N <= 9

    # Optional: Print classification report - THIS WILL LIKELY FAIL or be MISLEADING
    # because labels in y_test (0-4) don't cover all possible predictions (0-8)
    try:
        print('\nClassification Report (Attempting - likely problematic):')
        # Define target names based on TRUE classes (1-5 stars)
        target_names_actual = [str(i) for i in range(1, actual_num_classes + 1)] # Labels '1' through '5'
        # Use labels present in y_test for the report
        report = classification_report(y_test, y_pred_classes, labels=np.unique(y_test), target_names=target_names_actual, zero_division=0)
        print(report)
    except ValueError as e:
        print(f"\nCould not generate classification report due to label mismatch: {e}")

except Exception as e:
    print(f"\nError generating confusion matrix or report: {e}")



Evaluating model on the test set...
Test Loss: 0.6255
Test Accuracy (Keras evaluate): 0.7711

Generating predictions...
494/494 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step

--- Evaluation Warning ---
The model was trained with 9 output neurons as requested.
True labels (y_test) are in the range 0-4.
Predicted classes (y_pred_classes) are in the range 0-8.
Direct comparison using metrics like classification report or standard confusion matrix might be misleading.
--------------------------

Sklearn Accuracy Score (y_test vs y_pred_classes 0-8): 0.7711

Confusion Matrix (y_test [0-4] vs y_pred_classes [0-8]):
[[11538   160   713    92  1406]
 [ 2685   865  1095   179   955]
 [ 1516   131  2887   833  2020]
 [  513    40  1009  3592  6241]
 [ 1031    29   643  1317 57274]]
Confusion Matrix Shape: (5, 5)

Classification Report (Attempting - likely problematic):
              precision    recall  f1-score   support

           1       0.67      0.83      0.74     13909
           2       0.71      